Importing the libraries.

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql.window import Window
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType
from pyspark.dbutils import DBUtils

The code block below creates the catalog and schemas for our solution. 

The approach utilises a multi-hop data storage architecture (medallion), consisting of bronze, silver, and gold schemas within a 'streaming' catalog. 

In [0]:
# Databricks notebook source


# COMMAND ----------

catalog_name = "streaming1"
db_name = "bronze"
table_name='shopify_orders'
eventHubName1 = "streamingeventhubs"
key_vault='testScope'
connector='testsecrettyler'



dbutils.widgets.dropdown("trigger_available_now", "False", ["True", "False"])
trigger_available_now = dbutils.widgets.get("trigger_available_now") == "True"

notebook_name = DBUtils(spark).notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get().split("/")[-1].split(".")[0]
checkpoint_path = f"/Volumes/{catalog_name}/{db_name}/checkpoints/{notebook_name}/"




In [0]:
try:
    spark.sql(f"create catalog {catalog_name} managed location 'abfss://streamingdata-demo@dataengineerdemoweather.dfs.core.windows.net/';")
except:
    print('check if catalog already exists')

check if catalog already exists


In [0]:

try:
    spark.sql(f"create schema if not exists {catalog_name}.{db_name} ;") 
except:
    print('check if bronze schema already exists')

try:
    spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{db_name}.checkpoints;") 
except:
    print('check if bronze checkpoints already exists')




#### Bronze Layer

Set up Azure Event hubs connection string.

In [0]:
# Config
# Replace with your Event Hub namespace, name, and key
connectionString = dbutils.secrets.get(key_vault,connector)
eventHubName =eventHubName1


ehConf = {
  'eventhubs.connectionString' : sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString),
  'eventhubs.eventHubName': eventHubName
}

Reading and writing the stream to the bronze layer.

In [0]:
# Reading stream: Load data from Azure Event Hub into DataFrame 'df' using the previously configured settings
df = spark.readStream \
    .format("eventhubs") \
    .options(**ehConf) \
    .load() \

# Displaying stream: Show the incoming streaming data for visualization and debugging purposes
df.display()

# Writing stream: Persist the streaming data to a Delta table 'streaming.bronze.weather' in 'append' mode with checkpointing
df.writeStream\
    .option("checkpointLocation", checkpoint_path)\
    .outputMode("append")\
    .format("delta")\
    .toTable(f"{catalog_name}.{db_name}.{table_name}")

body,partition,offset,sequenceNumber,enqueuedTime,publisher,partitionKey,properties,systemProperties
eyJpZCI6IDYxNDU4NjM3NDU4MzQsICJjYW5jZWxfcmVhc29uIjogbnVsbCwgImNhbmNlbGxlZF9hdCI6IG51bGwsICJjaGVja291dF9pZCI6IDM3NjgyNjI3ODA1NDgyLCAiY3JlYXRlZF9hdCI6ICIyMDI0LTA5LTA4VDIxOjU= (truncated),0,12884913072,72,2024-09-09T01:51:56.782Z,null,null,Map(),Map(x-opt-sequence-number-epoch -> -1)
